In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import*
from ipynb.fs.full.TEST_PERFORMANCE import *
from ipynb.fs.full.ANN_CLASSIFICATION import *
import pygame
import wave
import librosa
import scipy.io.wavfile as wav
from scipy.io import wavfile
from ipynb.fs.full.MFCC import *
from sklearn.externals import joblib
from PIL import ImageTk,Image
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tkinter import font
from PIL import Image, ImageTk

Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class MyApp(object):
    
    def __init__(self,parent):
        self.root=parent
        self.frame=Frame(parent)
        image_path= 'YTU.jpg'
        width, height = 890,700
        canvas = tk.Canvas(self.frame, width=width, height=height)
        canvas.pack()

        img = ImageTk.PhotoImage(Image.open(image_path).resize((width, height), Image.ANTIALIAS))
        canvas.background = img  # Keep a reference in case this code is put in a function.
        bg = canvas.create_image(0, 0, anchor=tk.NW, image=img)
        
        title = tk.Label(self.frame, text="Development of Children's Speech Based Gender Classification System") 
        title.configure(fg="blue",font=("Helvetica", 18))
        label_window = canvas.create_window(50,50,anchor=tk.NW, window=title)
        
        title1 = tk.Label(self.frame, text=" using MFCC Features")
        title1.configure(fg="blue",font=("Helvetica", 18))
        label_window = canvas.create_window(300,100,anchor=tk.NW, window=title1)

        bFont = font.Font(family="Helvetica", size=16)
        button = tk.Button(self.frame, text="Classification",command=self.showFrame,font=bFont,fg="blue",height=1,width=20)
        button_window = canvas.create_window(200, 300, anchor=tk.NW, window=button)
        
        canvas.create_text(470, 320, text = '&', font = bFont, fill='blue')
        
        button1 = tk.Button(self.frame, text="Performance Analysis",command=self.openFrame,font=bFont,fg="blue",height=1,width=20)
        button1_window = canvas.create_window(500, 300, anchor=tk.NW, window=button1)
        
        button2 = tk.Button(self.frame, text="Exit",command=self.exitMain,font=bFont,fg="blue",height=1,width=10)
        button2_window = canvas.create_window(50, 650, anchor=tk.NW, window=button2)
        
        canvas.create_text(700, 650, text = 'Developed by Khin Aye Chan', font = bFont, justify = 'right', fill='blue')
       
        self.frame.pack()
       
        
    def hide(self):
        self.root.withdraw()
    
    def exitMain(self):
        self.root.withdraw()
        self.root.quit()
        
    def onCloseOtherFrame(self,otherFrame):
        otherFrame.destroy()
        self.show()
        
    def show(self):
        self.root.update()
        self.root.deiconify()
        
    def openFrame(self):
        self.hide()
        helv36 = font.Font(family='Helvetica', size=12)
        global content
        global chooseModel
        firstFrame=Toplevel()
        firstFrame.geometry("890x680")
        firstFrame.title("Performance Analysis")
        firstFrame.resizable(False,False)

        viewerPane = Frame(firstFrame)
        viewerPane.grid(row=0,column=0,rowspan=13,columnspan=8,sticky="NESW")
        viewerPane.grid_columnconfigure(0, weight=1)
        viewerPane.grid_rowconfigure(0, weight=1)

        actionPane = Frame(firstFrame)
        actionPane.grid(row=14,column=0,rowspan=33,columnspan=8,sticky="NESW")

        myLabel1 = Label(viewerPane, text="Split the DataSet!",fg="blue",font=helv36,width=55,height=3,anchor="w")
        myLabel2 = Label(viewerPane, text="Training %",fg="blue",font=helv36,width=55,height=3,anchor="w")
        myLabel1.grid(row=1, column=0)
        myLabel2.grid(row=1, column=1)

        entryTrain = tk.Entry(viewerPane,font=helv36,width=10)
        entryTrain.grid(column=1, row=1, padx=4, pady=5,ipady=3,sticky="S")

        myLabel3 = Label(viewerPane, text = "Choose Classification Model!",fg="blue",font=helv36,width=55,height=3,anchor="w")
        myLabel3.grid(row=3,column=0)

        combostyle = ttk.Style()
        comboModel = ttk.Combobox(viewerPane, values=[
                                    "Artificial Neural Network", 
                                    "Support Vector Machine",
                                    "Random Forest",
                                    "Logistic Regression",
                                    "Naive Bayes Classifier"],font=helv36,width=30)
        comboModel.grid(row=3,column=1,padx=10, pady=10,sticky=W)
        comboModel.current(0)
        
        
        
        button=Button(viewerPane, text="OK",fg='blue',command=lambda: result(),height=1,width=10,font=helv36)
        button.grid(column=0,row=5,sticky=tk.E)
        
        handler=lambda:self.onCloseOtherFrame(firstFrame)
        button1=Button(viewerPane,text="Back",fg='blue',command=handler,height=1,width=10,font=helv36)
        button1.grid(column=1,row=5,sticky=tk.E)

        ttk.Separator(viewerPane, orient=HORIZONTAL).grid(column=0,columnspan=2,row=10,sticky="we")

        
        
        def result():
            
            content = entryTrain.get()
            chooseModel=comboModel.get()
            trainsize=int(content)
            testsize=1-(trainsize/100)
          
            if chooseModel =='Random Forest':
                train,test,confusion,report,error=RF(testsize)
                display(train,test,confusion,report,error)
                
            elif chooseModel =='Artificial Neural Network':
                train,test,confusion,report,error=ANN(testsize)
                display(train,test,confusion,report,error)
        
            elif chooseModel =='Support Vector Machine':
                train,test,confusion,report,error=SVM(testsize)
                display(train,test,confusion,report,error)
        
            elif chooseModel =='Logistic Regression':
                train,test,confusion,report,error=LR(testsize)
                display(train,test,confusion,report,error)
        
            else:
                train,test,confusion,report,error=GNB(testsize)
                display(train,test,confusion,report,error)
            
            
        def display(train,test,matrix,report,error):
            trainLabel1 = Label(actionPane,text = "Model Accuracy for Training",height=3,width=55,font=helv36, anchor="w")
            trainLabel1.grid(column=0, row=13,sticky=tk.E)
            trainLabel2=Label(actionPane,text=str(train)+" %",height=3,width=55,font=helv36,anchor="w")
            trainLabel2.grid(column=1,row=13,sticky=tk.W)
            
            testLabel1 = Label(actionPane,text = "Model Accuracy for Testing",height=3,width=55,font=helv36,anchor="w")
            testLabel1.grid(column=0, row=17,sticky=tk.E)
            testLabel2=Label(actionPane,text=str(test)+" %",height=3,width=55,font=helv36,anchor="w")
            testLabel2.grid(column=1,row=17,sticky=tk.W)
            
            s1=Label(actionPane,text="",height=2,width=55,font=helv36,anchor="w")
            s1.grid(column=1,row=18,sticky=tk.W)
            
            
            matrixLabel1= Label(actionPane,text = "Confusion Matrix",height=3,width=55,font=helv36, anchor="w")
            matrixLabel1.grid(column=0,row=21,sticky=tk.E)
            matrixLabel2=Label(actionPane,text=matrix,height=5,width=55,font=helv36,anchor="w")
            matrixLabel2.grid(column=1,row=21,sticky=tk.W)
            
            s2=Label(actionPane,text="",height=1,width=55,font=helv36,anchor="w")
            s2.grid(column=1,row=24,sticky=tk.W)
            
            matrixLabel3=Label(actionPane,text="0=female",height=1,width=55,font=helv36,anchor="w")
            matrixLabel3.grid(column=1,row=30,sticky=tk.W)
            matrixLabel4=Label(actionPane,text="1=male",height=1,width=55,font=helv36,anchor="w")
            matrixLabel4.grid(column=1,row=35,sticky=tk.W)
            
            s3=Label(actionPane,text="",height=3,width=55,font=helv36,anchor="w")
            s3.grid(column=1,row=37,sticky=tk.W)
            
            reportLabel1= Label(actionPane,text = "Classification Report",height=3,width=55,font=helv36,anchor="w")
            reportLabel1.grid(column=0,row=40,sticky=tk.E)
            reportLabel2=Label(actionPane,text=report,height=7,width=55,font=helv36,anchor="w")
            reportLabel2.grid(column=1,row=40,sticky=tk.W)
            
            s4=Label(actionPane,text="",height=1,width=55,font=helv36,anchor="w")
            s4.grid(column=1,row=45,sticky=tk.W)
            
            errorLabel1= Label(actionPane,text = "Error Rate/Missclassification Rate",height=3,width=55,font=helv36,anchor="w")
            errorLabel1.grid(column=0,row=46,sticky=tk.E)
            errorLabel2=Label(actionPane,text=str(error)+" %",height=3,width=55,font=helv36,anchor="w")
            errorLabel2.grid(column=1,row=46,sticky=tk.W)
            
            
            
    def showFrame(self):
        self.hide()
        helv36 = font.Font(family='Helvetica', size=12)
        secondFrame=Toplevel()
        secondFrame.title("Classification")
        secondFrame.geometry("890x680")
        secondFrame.resizable(False,False)
        
        title = Label(secondFrame, text="What is Your Voice Gender? ",height=3,width=55,font=helv36)
        title.grid(column=1, row=1,padx=5, pady=5)

        button1=Button(secondFrame, text="Browse",fg='blue',command=lambda: chooseFile(),height=3,width=20,font=helv36)
        button1.grid(column=0, row=3,padx=5, pady=5)
 
        button2=Button(secondFrame, text="Play",fg='blue',command=lambda: masker_screen(),height=3,width=20,font=helv36)
        button2.grid(column=0, row=5)

        playLabel= Label(secondFrame, text="Play the chosen wave file! ",height=3,width=55,font=helv36)
        playLabel.grid(column=1, row=5,padx=5, pady=5)

        button3=Button(secondFrame, text="Feature Extraction",fg='blue',command=lambda: featureExtraction(),height=3,width=20,
                       font=helv36)
        button3.grid(column=0, row=7,padx=5, pady=5)
        l1= Label(secondFrame, text="",fg="blue",height=3,width=100,font=helv36)
        l1.grid(column=0, row=8,padx=5, pady=5,rowspan=1,columnspan=3)
        
        button4=Button(secondFrame, text="Classification",fg='blue',command=lambda: classifyGender(),height=3,width=20,
                       font=helv36)
        button4.grid(column=0, row=15,padx=5, pady=5)
        l2= Label(secondFrame, text="",fg="blue",height=3,width=100,font=helv36)
        l2.grid(column=0, row=16,padx=5, pady=5,rowspan=1,columnspan=3)
        
        
        handler=lambda:self.onCloseOtherFrame(secondFrame)
        button5=Button(secondFrame,text="Back",fg='blue',command=handler,font=helv36,width=20,height=3)
        button5.grid(column=0,row=16)     
        
        def chooseFile():
            global audio_file_path
            audio_file_path = filedialog.askopenfilename(filetypes=(("Audio Files", ".wav"), ("All Files", "*.*")))
            filename = Label(secondFrame,text = audio_file_path,height=3,width=75,font=helv36)
            filename.grid(row=3,column=1,columnspan=2)
        
        def masker_screen():  
            m_screen = Toplevel()
            m_screen.geometry('200x150')
            m_screen.title("");
            m_screen.transient(secondFrame) 
            m_label = Label(m_screen, text = "Playing Sound",height=3,width=100,font=helv36)
            m_label.pack(anchor = CENTER)

            if audio_file_path: # play sound if just not an empty string
                file_wav = wave.open(audio_file_path)
                frequency = file_wav.getframerate()
                pygame.mixer.init(frequency=frequency)
                pygame.mixer.music.load(audio_file_path)
                pygame.mixer.music.play()
                
        def featureExtraction():
            global write_feat
            helv36 = font.Font(family='Helvetica', size=12)
            write_mfeatures=[]
            sampling_rate,speech_signal = wavfile.read(audio_file_path)
            mfcc_feat=compute_mfcc(speech_signal,sampling_rate)
            write_feat=mfcc_feat.tolist()[0]
            fname= Label(secondFrame, text="MFCC Features",height=3,width=55,font=helv36)
            fname.grid(column=1, row=7,padx=5, pady=5)
            T = tk.Text(secondFrame, height=6, width=80,font=helv36,padx=15,pady=15,spacing2=10)
            T.insert(tk.END, str(write_feat))
            
            T.grid(column=1, row=8,rowspan=3,columnspan=2)
           
            l3= Label(secondFrame, text="",fg="blue",height=3,width=100,font=helv36)
            l3.grid(column=0, row=13,padx=5, pady=5,rowspan=1,columnspan=3)
        
        def classifyGender():
            helv36 = font.Font(family='Helvetica', size=12)
            loaded_model = joblib.load('RF_model.sav')
            label1= loaded_model.predict([write_feat])
            if label1[0] == 1:
                gender1='male'
            else:
                gender1='female'
                
            min_max_scaler = preprocessing.MinMaxScaler()
            x=np.array(write_feat)
            X_scale = min_max_scaler.fit_transform(x.reshape(12,-1))
            X_scale=X_scale.reshape(-1,12)
            loaded_model = joblib.load('ANN_model.sav')
            y_pred_class= loaded_model.predict([X_scale])
            label2= [ 1 if y>=0.5 else 0 for y in y_pred_class]
            if label2[0] ==1:
                gender2='male'
            else:
                gender2='female'
            
            
            loaded_model = joblib.load('LR_model.sav')
            label3= loaded_model.predict([write_feat])
            if label3[0] ==1:
                gender3='male'
            else:
                gender3='female'
                
                
            loaded_model = joblib.load('SVM_model.sav')
            label4= loaded_model.predict([write_feat])
            if label4[0] ==1:
                gender4='male'
            else:
                gender4='female'
           
            
            loaded_model = joblib.load('GNB_model.sav')
            label5= loaded_model.predict([write_feat])
            if label5 [0]==1:
                gender5='male'
            else:
                gender5='female'
                
            style = ttk.Style()
            style.configure("mystyle.Treeview", highlightthickness=0, bd=0, font=helv36) # Modify the font of the body
            style.configure("mystyle.Treeview.Heading", font=helv36) # Modify the font of the headings
            style.configure('mystyle.Treeview', rowheight=20)
            style.configure('mystyle.Treeview', columnwidth=15)
            cols = ( 'Classifiers', 'Predicted Output')
            listBox = ttk.Treeview(secondFrame, style="mystyle.Treeview",columns=cols, show='headings',height='6')
            for col in cols:
                listBox.heading(col, text=col)    
            
            listBox.grid(row=15, column=1, columnspan=2)
            tempList = [['Random Forest', gender1], ['Artificial Neural Network',gender2], ['Logistic Regression', gender3],
            ['Support Vector Machine', gender4],['Naive Bayes Classifier',gender5]]
            listBox.column("Classifiers", width=250, stretch=tk.NO)
            listBox.column("Predicted Output", width=150, stretch=tk.NO)
            for i,(name, score) in enumerate(tempList, start=1):
                listBox.insert( "","end",values= (name, score))
            
               
            listBox.columnconfigure(2, weight=2) # column with treeview
            listBox.rowconfigure(2, weight=2) #
           

In [ ]:
if __name__=="__main__":
    root=Tk()
    root.geometry("890x700")
    root.resizable(False, False)
    root.title("")
    app=MyApp(root)
    root.mainloop()